In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, desc

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

## Measurement Analysis

Design a query to retrieve the last 12 months of precipitation data and plot the results

In [ ]:
#determine column names and types:
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# Calculate the date 1 year ago from the last data point in the database
#first determine what the last date in the table is:
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

In [ ]:
query_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print("One year prior: ",query_date)

In [ ]:
# Perform a query to retrieve the data and precipitation scores

results = session.query(Measurement.date,Measurement.prcp).\
    filter(Measurement.date >= '2016-08-23').\
    order_by(Measurement.date).all()

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
prcp_df = pd.DataFrame(results)
prcp_df.rename(columns={'prcp':'precipitation'}, inplace=True)
prcp_df.set_index('date', inplace=True)
prcp_df.head(10)

In [ ]:
# Sort the dataframe by date
prcp_df.sort_index(inplace=True)
prcp_df

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
prcp_df.plot(figsize=(8,8), title = "Precipitation Analysis 08.23.16 - 08.23.17")
plt.xlabel("Date")
plt.ylabel("Inches")
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcp_df.describe()

## Station Analysis



In [ ]:
#Design a query to show how many stations are available in this dataset?
locations = session.query(Measurement).group_by(Measurement.station).count()
print("Number of stations available in this dataset: ", locations)

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).\
    order_by(desc(func.count(Measurement.station))).all()


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
res = session.query(func.min(Measurement.tobs),func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
              filter(Measurement.station == 'USC00519281').all()
temps = list(np.ravel(res))
cats = ["Lowest recorded temp: ", "Highest recorded temp: ", "Average recorded temp: "]

temp_cats = dict(zip(cats, temps))

temp_cats
    


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temp_freq = session.query(Measurement.tobs).\
    filter(Measurement.date >= '2016-08-23').\
    filter(Measurement.station == 'USC00519281').all()

temp_df = pd.DataFrame(temp_freq)
temp_df.plot.hist(title = "Temperature frequency of Station USC00519281 from 08.23.16 - 08.23.17", figsize = (8,5), bins=12)
plt.xlabel("Temperature")
plt.tight_layout()
plt.show()